# Análisis de Negocio — Proyecto ML en Salud Cardiometabólica

Este cuadernillo documenta el paso **Business Understanding** del EDA utilizando dos datasets:
- `cardio_data_processed.csv`
- `smoking_driking_dataset_Ver01.csv`

Se definen objetivos, casos de uso de ML, métricas de éxito y criterios de valor, y se realiza una exploración inicial para orientar el EDA técnico posterior.

## 1. Introducción

### 1.1 Objetivos del Análisis
- Comprender la estructura y relaciones potenciales entre los datasets clínicos y conductuales.
- Identificar variables clave candidatas para casos de uso de predicción/estratificación de riesgo.
- Definir métricas de éxito (negocio y técnicas) y valor esperado.
- Establecer un plan de implementación de alto nivel y riesgos principales.

## Hipótesis:
- **Clasificación binaria:** ¿Podemos predecir si un individuo tiene (o tendrá) enfermedad cardiovascular en base a sus factores de riesgo?
- **Regresión continua:** ¿Podemos estimar el valor esperado de un factor cuantitativo de riesgo, como presión arterial sistólica, nivel de colesterol o consumo de alcohol, a partir de otras variables del perfil del paciente?

## 2. Configuración del Entorno

In [12]:
# Cargar extensión de Kedro (solo hace falta una vez al inicio del notebook)
%load_ext kedro.ipython

# Recargar contexto y catálogo de Kedro
%reload_kedro

The kedro.ipython extension is already loaded. To reload it, use:
  %reload_ext kedro.ipython


[09/15/25 22:53:42] INFO     Resolved project path as: C:\Users\Ella es                             __init__.py:180
                             ella\Documents\kedro_projects\ml-ev1.                                                 
                             To set a different path, run '%reload_kedro <project_root>'                           

[09/15/25 22:53:43] INFO     Kedro is sending anonymous usage data with the sole purpose of improving plugin.py:243
                             the product. No personal data or IP addresses are stored on our side. To              
                             opt out, set the `KEDRO_DISABLE_TELEMETRY` or `DO_NOT_TRACK` environment              
                             variables, or create a `.telemetry` file in the current working                       
                             directory with the contents `consent: false`. To hide this message,                   
                             explicitly grant or deny consent. Read more at                                        
                             https://docs.kedro.org/en/stable/configuration/telemetry.html                         

[09/15/25 22:53:44] INFO     Kedro project ml_ev1                                                   __init__.py:146

                    INFO     Defined global variable 'context', 'session', 'catalog' and            __init__.py:147
                             'pipelines'                                                                           

In [9]:
# Importación de librerías necesarias
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

print("Entorno listo.")

Entorno listo.


## 3. Carga y Exploración de Datos
### 3.1 Carga de Datasets

In [13]:
FILES = ["cardio", "lifestyle"]

datasets = {}
print("Cargando datasets desde el catálogo de Kedro...\n")

for name in FILES:
    try:
        datasets[name] = catalog.load(name)
        print(f"O Dataset '{name}' cargado ({datasets[name].shape[0]} filas, {datasets[name].shape[1]} columnas)")
    except Exception as e:
        print(f"X Error cargando dataset '{name}': {e}")

print(f"\nTotal de datasets cargados: {len(datasets)}")

Cargando datasets desde el catálogo de Kedro...



[09/15/25 22:53:56] INFO     Loading data from cardio (CSVDataset)...                          data_catalog.py:1046

O Dataset 'cardio' cargado (68205 filas, 17 columnas)


                    INFO     Loading data from lifestyle (CSVDataset)...                       data_catalog.py:1046

O Dataset 'lifestyle' cargado (991346 filas, 24 columnas)

Total de datasets cargados: 2


### 3.2 Descripción de Tablas
Análisis descriptivo de cada dataset y su utilidad en el contexto de salud cardiometabólica.

In [15]:
dataset_descriptions = {
    'cardio': 'Variables clínicas/antropométricas (p. ej., presión arterial, lípidos, glucosa, IMC, edad, sexo)',
    'lifestyle': 'Conductas de salud (tabaquismo, alcohol, estilo de vida).'
}

print("DESCRIPCIÓN DE DATASETS" + "="*80)
for name, df in datasets.items():
    description = dataset_descriptions.get(name, '—')
    print(f"\n📊 {name.upper()}")
    print(f"   Descripción: {description}")
    print(f"   Dimensiones: {df.shape[0]:,} filas × {df.shape[1]} columnas")
    cols_preview = ', '.join(map(str, df.columns[:10]))
    print(f"   Columnas: {cols_preview}{'...' if len(df.columns) > 10 else ''}")

DESCRIPCIÓN DE DATASETS================================================================================

📊 CARDIO
   Descripción: Variables clínicas/antropométricas (p. ej., presión arterial, lípidos, glucosa, IMC, edad, sexo)
   Dimensiones: 68,205 filas × 17 columnas
   Columnas: id, age, gender, height, weight, ap_hi, ap_lo, cholesterol, gluc, smoke...

📊 LIFESTYLE
   Descripción: Conductas de salud (tabaquismo, alcohol, estilo de vida).
   Dimensiones: 991,346 filas × 24 columnas
   Columnas: sex, age, height, weight, waistline, sight_left, sight_right, hear_left, hear_right, SBP...


### 3.3 Análisis de Estructura de Datos

In [18]:
def analyze_dataset(df, name):
    """Análisis estructural y de calidad para un DataFrame."""
    print(f"\n{'-'*100}\nANÁLISIS DE: {name.upper()}\n{'-'*100}")
    print(f"\n📋 Información General:")
    print(f"   - Total de registros: {len(df):,}")
    print(f"   - Total de columnas: {len(df.columns)}")
    print(f"   - Memoria utilizada: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

    # Tipos de datos
    dtype_counts = df.dtypes.value_counts()
    print(f"\n📊 Tipos de Datos:")
    for dtype, count in dtype_counts.items():
        print(f"   - {dtype}: {count} columnas")

    # Nulos
    null_counts = df.isnull().sum()
    null_cols = null_counts[null_counts > 0]
    if len(null_cols) > 0:
        print(f"\n⚠️ Columnas con Valores Nulos:")
        for col, count in null_cols.sort_values(ascending=False).items():
            percentage = (count / len(df)) * 100 if len(df) else 0
            print(f"   - {col}: {count:,} ({percentage:.1f}%)")
    else:
        print(f"\n✓ No hay valores nulos en este dataset (o no se detectaron)")

    # Columnas
    print(f"\n🎲 Columnas del Dataset (primeras 30):")
    for i, col in enumerate(df.columns[:30], 1):
        dtype_str = str(df[col].dtype)
        print(f"   {i:2}. {col:40} ({dtype_str})")

    # Estadísticas numéricas
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    if len(numeric_cols) > 0:
        print(f"\n📈 Resumen Estadístico (numéricas):")
        print(df[numeric_cols].describe().round(2).to_string())
    else:
        print("\nℹ️ No se encontraron columnas numéricas para resumen estadístico.")

for dataset_name, df in datasets.items():
    analyze_dataset(df, dataset_name)


----------------------------------------------------------------------------------------------------
ANÁLISIS DE: CARDIO
----------------------------------------------------------------------------------------------------

📋 Información General:
   - Total de registros: 68,205
   - Total de columnas: 17
   - Memoria utilizada: 16.46 MB

📊 Tipos de Datos:
   - int64: 13 columnas
   - float64: 2 columnas
   - object: 2 columnas

✓ No hay valores nulos en este dataset (o no se detectaron)

🎲 Columnas del Dataset (primeras 30):
    1. id                                       (int64)
    2. age                                      (int64)
    3. gender                                   (int64)
    4. height                                   (int64)
    5. weight                                   (float64)
    6. ap_hi                                    (int64)
    7. ap_lo                                    (int64)
    8. cholesterol                              (int64)
    9. gluc        

## 4. Entendimiento del Negocio (Salud Cardiometabólica)
### 4.1 Contexto
Las enfermedades cardiovasculares y metabólicas representan una carga significativa para sistemas de salud. Modelos ML pueden apoyar estratificación de riesgo, priorización de pesquisa y asignación eficiente de recursos clínicos.

### 4.2 Stakeholders y Necesidades

In [22]:
stakeholders = pd.DataFrame({
    'Stakeholder': [
        'Clínicos (Medicina Familiar/Interna)',
        'Pacientes/Usuarios',
        'Gestión Hospitalaria/Clínicas',
        'Aseguradoras/Financiadores',
        'Salud Pública',
        'Equipo de Datos/Producto'
    ],
    'Necesidad Principal': [
        'Priorizar controles y derivaciones por riesgo',
        'Prevención personalizada y alertas comprensibles',
        'Optimizar cupos, tiempos y costos asistenciales',
        'Gestionar riesgo poblacional y costos futuros',
        'Monitorear factores de riesgo a nivel poblacional',
        'Sistemas reproducibles, auditables y escalables'
    ],
    'Valor del ML': [
        'Listas de trabajo por riesgo y reglas operativas',
        'Recomendaciones accionables y explicables',
        'Planificación basada en carga esperada',
        'Modelos de propensión y costo-efectividad',
        'Mapas de riesgo y vigilancia sindrómica',
        'Pipelines versionados, monitoreo de drift'
    ]
})
print("ANÁLISIS DE STAKEHOLDERS")
print("-"*135)
print(stakeholders.to_string(index=False))

ANÁLISIS DE STAKEHOLDERS
---------------------------------------------------------------------------------------------------------------------------------------
                         Stakeholder                               Necesidad Principal                                     Valor del ML
Clínicos (Medicina Familiar/Interna)     Priorizar controles y derivaciones por riesgo Listas de trabajo por riesgo y reglas operativas
                  Pacientes/Usuarios  Prevención personalizada y alertas comprensibles        Recomendaciones accionables y explicables
       Gestión Hospitalaria/Clínicas   Optimizar cupos, tiempos y costos asistenciales           Planificación basada en carga esperada
          Aseguradoras/Financiadores     Gestionar riesgo poblacional y costos futuros        Modelos de propensión y costo-efectividad
                       Salud Pública Monitorear factores de riesgo a nivel poblacional          Mapas de riesgo y vigilancia sindrómica
            Equipo de D

## 5. Definición de Casos de Uso de Machine Learning
Se proponen dos casos de uso genéricos, ajustables según disponibilidad de variables/labels.

### 5.1 Caso de Clasificación (binaria) — Riesgo Cardiometabólico Alto
**Objetivo:** Predecir probabilidad de pertenecer a un grupo de *alto riesgo* para priorizar intervención preventiva.

**Métrica Principal:** AUROC y/o PR-AUC.  
**Métricas Operativas:** PPV/NPV a umbrales, *Lift* en deciles altos, Brier score (calibración).  
**Valor:** Priorización de pacientes y asignación eficiente de recursos.

In [23]:
from typing import List

def find_candidate_label_columns(df) -> List[str]:
    candidates = []
    label_keys = ['target','label','outcome','y','event','cardio','cardiovascular','disease','risk','hypertension','diabetes']
    for col in df.columns:
        low = str(col).lower()
        if any(k in low for k in label_keys):
            candidates.append(col)
    return candidates

label_candidates = {}
for name, df in datasets.items():
    label_candidates[name] = find_candidate_label_columns(df)

print("POSIBLES COLUMNAS DE ETIQUETA (para clasificación):")
for name, cols in label_candidates.items():
    print(f" - {name}: {cols if cols else 'No se detectaron candidatos obvios'}")

POSIBLES COLUMNAS DE ETIQUETA (para clasificación):
 - cardio: ['cardio', 'age_years', 'bp_category', 'bp_category_encoded']
 - lifestyle: ['triglyceride', 'SMK_stat_type_cd', 'DRK_YN']


**Nota:** Si no existe una etiqueta explícita, se puede definir un label derivado (ej.: *hipertensión alta* si `systolic_bp ≥ 140` o *síndrome metabólico proxy* según umbrales clínicos acordados). Esta definición debe validarse con stakeholders clínicos para evitar *label leakage*.

### 5.2 Caso de Regresión — Score de Riesgo Continuo / Marcador Clínico
**Objetivo:** Estimar un **score continuo de riesgo** (o un marcador clínico, p. ej., presión sistólica) como indicador de severidad/urgencia.

**Métrica Principal:** MAE/RMSE.  
**Métricas Secundarias:** R², error absoluto relativo por subgrupo.  
**Valor:** Seguimiento longitudinal y objetivos personalizados.

In [24]:
# Detección de posibles objetivos continuos (marcadores clínicos) por nombre
def find_candidate_regression_targets(df) -> List[str]:
    keys = ['systolic','diastolic','sbp','dbp','chol','trig','hdl','ldl','glucose','hba1c','bmi','weight','waist']
    candidates = []
    for col in df.select_dtypes(include=[np.number]).columns:
        low = str(col).lower()
        if any(k in low for k in keys):
            candidates.append(col)
    return candidates

reg_targets = {}
for name, df in datasets.items():
    reg_targets[name] = find_candidate_regression_targets(df)

print("POSIBLES VARIABLES OBJETIVO PARA REGRESIÓN (clínicas continuas):")
for name, cols in reg_targets.items():
    print(f" - {name}: {cols if cols else 'No se detectaron candidatos por nombre'}")

POSIBLES VARIABLES OBJETIVO PARA REGRESIÓN (clínicas continuas):
 - cardio: ['weight', 'cholesterol', 'bmi']
 - lifestyle: ['weight', 'waistline', 'SBP', 'DBP', 'tot_chole', 'HDL_chole', 'LDL_chole', 'triglyceride']


## 6. Análisis de Calidad de Datos

In [27]:
print("EVALUACIÓN DE CALIDAD DE DATOS")
print("-"*100)

quality_metrics = []
for name, df in datasets.items():
    total_cells = df.shape[0] * df.shape[1]
    null_cells = int(df.isnull().sum().sum())
    completeness = ((total_cells - null_cells) / total_cells * 100) if total_cells > 0 else 0
    duplicate_rows = int(df.duplicated().sum()) if len(df) > 0 else 0

    quality_metrics.append({
        'Dataset': name,
        'Filas': len(df),
        'Columnas': df.shape[1],
        'Completitud_%': round(completeness, 1),
        'Duplicados': duplicate_rows,
        'Calidad': '✓ Alta' if completeness > 95 and duplicate_rows == 0 else
                   '⚠ Media' if completeness > 80 else '✗ Baja'
    })

quality_df = pd.DataFrame(quality_metrics)
print(quality_df.to_string(index=False))

high_quality = sum(1 for m in quality_metrics if '✓' in m['Calidad'])
medium_quality = sum(1 for m in quality_metrics if '⚠' in m['Calidad'])
low_quality = sum(1 for m in quality_metrics if '✗' in m['Calidad'])

print(f"\n Resumen de Calidad:")
print(f"   - Datasets de alta calidad: {high_quality}")
print(f"   - Datasets de calidad media: {medium_quality}")
print(f"   - Datasets de baja calidad: {low_quality}")

EVALUACIÓN DE CALIDAD DE DATOS
----------------------------------------------------------------------------------------------------
  Dataset  Filas  Columnas  Completitud_%  Duplicados Calidad
   cardio  68205        17        100.000           0  ✓ Alta
lifestyle 991346        24        100.000          26 ⚠ Media

 Resumen de Calidad:
   - Datasets de alta calidad: 1
   - Datasets de calidad media: 1
   - Datasets de baja calidad: 0


## 7. Variables Clave Identificadas (Candidatas)
**Nota:** Este listado es preliminar. La selección final debe validarse con criterio clínico y disponibilidad real en columnas.

In [28]:
key_vars = pd.DataFrame({
    'Categoría': [
        'Clínicas/Antropométricas','Clínicas/Antropométricas','Clínicas/Antropométricas',
        'Clínicas/Antropométricas','Clínicas/Antropométricas','Clínicas/Antropométricas',
        'Conductuales','Conductuales','Conductuales',
        'Demográficas','Demográficas'
    ],
    'Variable (candidata)': [
        'systolic_bp / diastolic_bp (si existen)',
        'glucose / HbA1c (si existen)',
        'lipids: LDL/HDL/TG/TC (si existen)',
        'BMI / weight / waist (si existen)',
        'renal/liver markers (si existen)',
        'comorbidities flags (si existen)',
        'smoking_status / intensity',
        'alcohol_frequency / units',
        'physical_activity / sleep (si existen)',
        'age',
        'sex'
    ],
    'Tipo': [
        'Numérica','Numérica','Numérica','Numérica','Numérica','Categórica/Binaria',
        'Categórica/Ordinal','Numérica/Ordinal','Categórica/Ordinal','Numérica','Categórica'
    ],
    'Uso Potencial': [
        'Objetivo regresión o feature',
        'Feature/criterio de riesgo',
        'Feature/estratificación',
        'Feature/estratificación',
        'Feature/riesgo adicional',
        'Interacciones/ajustes',
        'Feature conductual clave',
        'Feature conductual clave',
        'Features estilo de vida',
        'Ajuste y segmentación',
        'Ajuste y segmentación'
    ]
})
print(key_vars.to_string(index=False))

               Categoría                    Variable (candidata)               Tipo                Uso Potencial
Clínicas/Antropométricas systolic_bp / diastolic_bp (si existen)           Numérica Objetivo regresión o feature
Clínicas/Antropométricas            glucose / HbA1c (si existen)           Numérica   Feature/criterio de riesgo
Clínicas/Antropométricas      lipids: LDL/HDL/TG/TC (si existen)           Numérica      Feature/estratificación
Clínicas/Antropométricas       BMI / weight / waist (si existen)           Numérica      Feature/estratificación
Clínicas/Antropométricas        renal/liver markers (si existen)           Numérica     Feature/riesgo adicional
Clínicas/Antropométricas        comorbidities flags (si existen) Categórica/Binaria        Interacciones/ajustes
            Conductuales              smoking_status / intensity Categórica/Ordinal     Feature conductual clave
            Conductuales               alcohol_frequency / units   Numérica/Ordinal     Feature 

## 8. Plan de Implementación (Alto Nivel)
1. Cerrar **definición de label** (clínico-aceptable, sin leakage).  
2. Establecer **diccionario de variables** y reglas de limpieza.  
3. Pipeline reproducible de **feature engineering** (versionado).  
4. **Baselines** (logística/árboles) y evaluación por subgrupos.  
5. Calibración, curvas decisión (net benefit) y umbrales operativos.  
6. Documentación, *model card*, y plan de **monitoreo de drift**.

## 9. Riesgos y Mitigaciones
- **Leakage**: variables pos-evento o derivadas del outcome → revisión clínica + auditoría de features.
- **Desbalance de clases**: métrica enfocada en utilidad (PR-AUC, PPV por umbral), ponderación/muestreo.
- **Shift temporal**: validación temporal + monitoreo continuo.
- **Calidad de captura**: reglas de plausibilidad y *data contracts*.
- **Equidad**: métricas por subgrupo y ajuste de umbrales si procede.

## 10. Conclusiones y Próximos Pasos

In [30]:
print("CONCLUSIONES DEL ANÁLISIS DE NEGOCIO")
print("-"*100)
conclusions = [
    "Se dispone de datos clínicos y conductuales con potencial para predicción/estratificación.",
    "Los casos de uso (clasificación de alto riesgo, regresión de score/marcador) son viables condicionados a la definición clínica del label.",
    "El valor de negocio reside en priorizar intervención y optimizar recursos.",
    "Se identificaron variables candidatas y se estableció un plan de implementación de alto nivel."
]
for c in conclusions:
    print("\n• " + c)

print("\n" + "-"*100)
print("\nPRÓXIMOS PASOS INMEDIATOS:")
print("-"*100)
next_steps = [
    "1) Validar con stakeholders la definición del label (o reglas de derivación).",
    "2) Elaborar diccionario de datos y plan de limpieza/imputación.",
    "3) Construir baselines y comparar desempeño por subgrupos.",
    "4) Diseñar umbrales operativos y curvas de decisión.",
    "5) Documentar supuestos y riesgos en una model card."
]
for s in next_steps:
    print("\n" + s)

CONCLUSIONES DEL ANÁLISIS DE NEGOCIO
----------------------------------------------------------------------------------------------------

• Se dispone de datos clínicos y conductuales con potencial para predicción/estratificación.

• Los casos de uso (clasificación de alto riesgo, regresión de score/marcador) son viables condicionados a la definición clínica del label.

• El valor de negocio reside en priorizar intervención y optimizar recursos.

• Se identificaron variables candidatas y se estableció un plan de implementación de alto nivel.

----------------------------------------------------------------------------------------------------

PRÓXIMOS PASOS INMEDIATOS:
----------------------------------------------------------------------------------------------------

1) Validar con stakeholders la definición del label (o reglas de derivación).

2) Elaborar diccionario de datos y plan de limpieza/imputación.

3) Construir baselines y comparar desempeño por subgrupos.

4) Diseñar umbr